In [1]:
load_model = 'regnet_y_3_2gf_based_model2.pt'

data_dir = '../../../stanford_dogs_new/'

In [2]:
# import numpy as np

from IPython.display import Image

import os
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision.models as models
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

from torchvision.datasets import ImageFolder
from PIL import Image

from torch.autograd import Variable

from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy


## load the regnet_y_3_2gf based model

In [3]:
regnet_y_3_2gf_loaded = models.regnet_y_3_2gf(pretrained=True)

for param in regnet_y_3_2gf_loaded.parameters():
    param.requires_grad = False

regnet_y_3_2gf_loaded.fc = nn.Linear(in_features=1512, out_features=120, bias=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
regnet_y_3_2gf_loaded = regnet_y_3_2gf_loaded.to(device)



regnet_y_3_2gf_loaded.load_state_dict(torch.load('regnet_y_3_2gf_based_model2.pt'), strict=False)
regnet_y_3_2gf_loaded.eval()

RegNet(
  (stem): SimpleStemIN(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (trunk_output): Sequential(
    (block1): AnyStage(
      (block1-0): ResBottleneckBlock(
        (proj): ConvNormActivation(
          (0): Conv2d(32, 72, kernel_size=(1, 1), stride=(2, 2), bias=False)
          (1): BatchNorm2d(72, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (f): BottleneckTransform(
          (a): ConvNormActivation(
            (0): Conv2d(32, 72, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(72, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (b): ConvNormActivation(
            (0): Conv2d(72, 72, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=3, bias=False)
            (1): B

In [4]:
model = regnet_y_3_2gf_loaded

In [5]:
# preprocess the image for the model
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
    )
])

path = '../../../stanford_dogs_new/test'

list_subfolders_with_paths = [f.path for f in os.scandir(path) if f.is_dir()]

# gets the breed name from subfolders name like 'n02099429-curly-coated_retriever'
def rename(name):
    return ' '.join(' '.join(name.split('-')[1:]).split('_'))

per_class_accuracy = dict()
total_accuracy = 0

for breed_dir in list_subfolders_with_paths:
    
    breed = rename(breed_dir.split('/')[-1])
    
    images = os.listdir(breed_dir)
    
    matched = False
    matches = 0
    total_images = len(images)

    for image in images:

        input_image = Image.open(breed_dir + '/' + image)


        input_tensor = preprocess(input_image)
        if torch.cuda.is_available():
            input_tensor = Variable(input_tensor.cuda())

        input_batch = input_tensor.unsqueeze(0)
        out = model(input_batch)

        probabilities = torch.nn.functional.softmax(out[0], dim=0)
        # print(probabilities)

        with open("../../stanford_dogs_breeds_classes_final.txt", "r") as f:
            categories = [s.strip() for s in f.readlines()]

        predicted_breeds = []
        top1_prob, top1_catid = torch.topk(probabilities, 1)
        for i in range(top1_prob.size(0)):
            # predicted_breeds.append([categories[top3_catid[i]], top3_prob[i].item()*100])
            predicted_breed = categories[top1_catid[i]]

        # list to be used directly by the application (predicted_breed, probability)
        # print("predicted_breeds are: \n", predicted_breed)
        if breed == predicted_breed:
            matched = True
            matches = matches + 1

        per_class_accuracy[breed] = matches / total_images * 100

per_class_accuracy

{'Pomeranian': 90.9090909090909,
 'basset': 94.44444444444444,
 'Japanese spaniel': 100.0,
 'Bouvier des Flandres': 73.33333333333333,
 'German shepherd': 87.5,
 'Brabancon griffon': 100.0,
 'Blenheim spaniel': 100.0,
 'kelpie': 75.0,
 'Shih Tzu': 90.9090909090909,
 'curly coated retriever': 87.5,
 'Lakeland terrier': 40.0,
 'Irish setter': 100.0,
 'German short haired pointer': 93.75,
 'flat coated retriever': 87.5,
 'Irish water spaniel': 86.66666666666667,
 'Dandie Dinmont': 94.44444444444444,
 'clumber': 100.0,
 'Yorkshire terrier': 82.35294117647058,
 'schipperke': 100.0,
 'briard': 75.0,
 'miniature poodle': 56.25,
 'chow': 95.0,
 'Saluki': 90.0,
 'boxer': 81.25,
 'Pekinese': 86.66666666666667,
 'komondor': 93.75,
 'Brittany spaniel': 87.5,
 'black and tan coonhound': 81.25,
 'Appenzeller': 43.75,
 'Eskimo dog': 6.666666666666667,
 'giant schnauzer': 62.5,
 'Old English sheepdog': 100.0,
 'Chihuahua': 87.5,
 'miniature schnauzer': 87.5,
 'redbone': 86.66666666666667,
 'wire haire

In [6]:
total_accuracy = sum(per_class_accuracy.values())/120
total_accuracy

84.8690484495438

### Top 10 Breeds with least accuracy

In [7]:
k = 10
sorted(per_class_accuracy.items(), key=lambda x: x[1])[:k]

[('Eskimo dog', 6.666666666666667),
 ('collie', 37.5),
 ('Lakeland terrier', 40.0),
 ('Lhasa', 42.10526315789473),
 ('Appenzeller', 43.75),
 ('Walker hound', 50.0),
 ('toy terrier', 50.0),
 ('miniature poodle', 56.25),
 ('cairn', 60.0),
 ('giant schnauzer', 62.5)]

### Top 15 Breeds with most accuracy

In [8]:
k = 15
sorted(per_class_accuracy.items(), key=lambda x: x[1], reverse=True)[:k]

[('Japanese spaniel', 100.0),
 ('Brabancon griffon', 100.0),
 ('Blenheim spaniel', 100.0),
 ('Irish setter', 100.0),
 ('clumber', 100.0),
 ('schipperke', 100.0),
 ('Old English sheepdog', 100.0),
 ('Pembroke', 100.0),
 ('keeshond', 100.0),
 ('Samoyed', 100.0),
 ('malinois', 100.0),
 ('Rottweiler', 100.0),
 ('West Highland white terrier', 100.0),
 ('EntleBucher', 100.0),
 ('Doberman', 100.0)]